<a target="_blank" href="https://colab.research.google.com/github/echosprint/TabularTransformer/blob/main/income_analysis.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

---

**See also** [RTDL](https://github.com/yandex-research/rtdl)
-- **other projects on tabular deep learning**.

---

- This notebook provides a usage example of the
  [TabularTransformer](https://github.com/echosprint/TabularTransformer)
  package.
- Hyperparameters are not tuned and may be suboptimal.

In [ ]:
%pip install git+https://github.com/echosprint/TabularTransformer.git

In [ ]:
import tabular_transformer as ttf
import pandas as pd
import torch

In [ ]:
income_dataset_path = ttf.prepare_income_dataset()

In [ ]:

class IncomeDataReader(ttf.DataReader):
    ensure_categorical_cols = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country', 'income']
    ensure_numerical_cols = ['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss', 'hours.per.week']

    def read_data_file(self, file_path):
        df = pd.read_csv(file_path)
        return df

In [ ]:
income_reader = IncomeDataReader(income_dataset_path)
df = income_reader.read_data_file()
df.head(3)

In [ ]:
split = income_reader.split_data({'test': 0.2, 'train': -1})
print(split)

In [ ]:
# examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1', etc.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dtype = 'bfloat16' if torch.cuda.is_available() \
            and torch.cuda.is_bf16_supported() else 'float16'  # 'float32' or 'bfloat16' or 'float16'

ts = ttf.TrainSettings(wandb_log=False, 
                       device=device, 
                       dtype=dtype, 
                       eval_iters=20)

tp = ttf.TrainParameters(train_epochs=15,
                         batch_size=128,
                         eval_interval=100,
                         warmup_iters=100,
                         validate_split=0.2)

hp = ttf.HyperParameters(dim=64,
                         n_layers=6)

trainer = ttf.Trainer(hp=hp, ts=ts)
trainer.train(data_reader=IncomeDataReader(split['train']), tp=tp, resume=False)


In [ ]:
predictor = ttf.Predictor(checkpoint='out/ckpt.pt')
predictor.predict(data_reader=IncomeDataReader(split['test']),
                  save_as="prediction_output.csv")